In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
#import data as d
#import model
import time
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc

from util import *


In [3]:
path = './stock_220430'
path_asset = './stock_220430/asset'
INDEX_DATA = make_index_data(start_date='2012-01-01',end_date="2022-05-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
all_stock = StockData_3(path_asset,
                            INDEX_DATA,
                            25,
                            70,
                            30,
                            30,
                            0.8,
                            1024,
                            2000,
                            20)

period : 2014-05-21 00:00:00 ~ 2022-04-28 00:00:00
StockPrice shape:  (2042, 84)


making dataset progress: 100%|█| 1953/1953 [03:37<


In [22]:
INDEX_DATA.columns.to_list()[1:]

['SP500',
 'Dow',
 'Nasdaq',
 'CrudeOil',
 'Gold',
 'Silver',
 'EURUSD',
 'gsci',
 'vix']

In [24]:
total_list = all_stock.include_asset
total_list.extend(INDEX_DATA.columns.to_list()[1:])
total_list

['ABBV',
 'ACN',
 'AEP',
 'AIZ',
 'ALLE',
 'AMAT',
 'AMP',
 'AMZN',
 'AVB',
 'AVY',
 'AXP',
 'BDX',
 'BF-B',
 'BMY',
 'BR',
 'CDW',
 'CE',
 'CHTR',
 'CNC',
 'CNP',
 'COP',
 'CTAS',
 'DG',
 'DPZ',
 'DRE',
 'DXC',
 'FB',
 'GOOG',
 'GPC',
 'HIG',
 'HST',
 'JPM',
 'KR',
 'PG',
 'PPL',
 'PRU',
 'RE',
 'ROL',
 'ROST',
 'UNH',
 'URI',
 'V',
 'VRSK',
 'XOM',
 'IVV',
 'IWM',
 'EWU',
 'EWG',
 'EWL',
 'EWQ',
 'IEUS',
 'EWJ',
 'EWT',
 'MCHI',
 'INDA',
 'EWY',
 'EWA',
 'EWH',
 'EWZ',
 'EWC',
 'IEMG',
 'LQD',
 'HYG',
 'SHY',
 'IEF',
 'TLT',
 'SEGA.L',
 'IAU',
 'SLV',
 'GSG',
 'ICLN',
 'IXN',
 'IGF',
 'SPMV.L',
 'MVEU.L',
 'XLK',
 'XLF',
 'XLV',
 'XLE',
 'XLY',
 'XLI',
 'XLU',
 'XLP',
 'XLB',
 'SP500',
 'Dow',
 'Nasdaq',
 'CrudeOil',
 'Gold',
 'Silver',
 'EURUSD',
 'gsci',
 'vix']

In [12]:
i = 50
priceSet = all_stock.stockPrice.loc[i-50:i-1].copy()

positive=[]
negative=[] 
corr=priceSet[-50:].corr(method='pearson')

for j in corr.columns:
    tempCorr=corr[j].sort_values(ascending=False)
    index_P=tempCorr[1:30+1].index
    index_N=tempCorr[-30:].index        
    
    priceSet=priceSet[-25:]
    posSet=priceSet[index_P]
    negSet=priceSet[index_N]
    posSet.columns=range(30)
    negSet.columns=range(30)
    
    positive.append(posSet)
    negative.append(negSet)

In [13]:
corr

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,1.000000,-0.020815,0.443369,-0.321805,0.396984,0.453772,0.428456,0.202631,0.084452,0.537717,...,0.285845,0.242932,0.402541,0.554359,0.402318,0.590306,0.252317,0.408070,0.353422,0.450196
1,-0.020815,1.000000,0.185165,0.715849,0.467135,0.141241,-0.117412,-0.235132,-0.475813,0.215461,...,0.184767,-0.312977,0.129265,-0.241998,0.012851,-0.103068,0.617522,0.279039,0.156772,0.004659
2,0.443369,0.185165,1.000000,-0.173598,0.517438,0.668862,0.629366,0.320163,0.365617,0.736596,...,0.282474,0.443191,0.652770,0.603066,0.687818,0.620973,0.389688,0.940623,0.616338,0.702698
3,-0.321805,0.715849,-0.173598,1.000000,-0.010115,-0.283897,-0.569656,-0.431143,-0.628488,-0.139309,...,-0.269082,-0.630330,-0.342865,-0.644022,-0.530310,-0.483046,0.434621,-0.115664,-0.045344,-0.420186
4,0.396984,0.467135,0.517438,-0.010115,1.000000,0.810654,0.547002,0.443815,-0.109312,0.774587,...,0.598809,0.273196,0.690611,0.374881,0.713321,0.536005,0.748259,0.602063,0.642966,0.518104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0.590306,-0.103068,0.620973,-0.483046,0.536005,0.748384,0.937013,0.749320,0.628994,0.670912,...,0.583947,0.852191,0.908026,0.902826,0.769111,1.000000,0.406220,0.455043,0.715334,0.914268
80,0.252317,0.617522,0.389688,0.434621,0.748259,0.589238,0.303457,0.406970,-0.272625,0.691438,...,0.424837,0.075619,0.527597,0.105697,0.342612,0.406220,1.000000,0.401941,0.678893,0.379527
81,0.408070,0.279039,0.940623,-0.115664,0.602063,0.617656,0.505409,0.208367,0.180209,0.735163,...,0.296774,0.280448,0.558267,0.460583,0.685499,0.455043,0.401941,1.000000,0.538741,0.574739
82,0.353422,0.156772,0.616338,-0.045344,0.642966,0.782287,0.644061,0.633054,0.224461,0.812548,...,0.407146,0.478065,0.700165,0.542738,0.632448,0.715334,0.678893,0.538741,1.000000,0.673501
